In [1]:
import codenamesLLM
import pandas as pd
import openpyxl
from tqdm import tqdm

c:\Users\miche\OneDrive\Desktop\Codenames Progetto PDE\Codenames-LLM\codenames_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Path to your Excel file
file_path = "experiment_data\model_tournament\model_tournament_mich1.xlsx" #compy the model_tournament_input to use one new

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Iterate through the rows using titled columns
for index, row in tqdm(df.iterrows(), total=len(df)):
    # Access values using column titles
    red_team = row['red_model']  # Replace with the actual column title for the red team
    blue_team = row['blue_model']  # Replace with the actual column title for the blue team
    already_played = pd.notna(row['winner'])
    
    if (pd.notna(red_team) and pd.notna(blue_team)) and not(already_played):  # Check if both values are not NaN
        try:
            print(f"Playing {red_team} vs {blue_team}...")
            # Call your function and get the result
            result = codenamesLLM.play_game(red_model = red_team, blue_model = blue_team)

            red_stats = codenamesLLM.analyze_team_guesses(result[3], "RED")
            blue_stats = codenamesLLM.analyze_team_guesses(result[3], "BLUE")

            df.at[index, 'red_model'] = red_team
            df.at[index, 'red_model'] = red_team
            df.at[index, 'winner'] = result[0]  
            df.at[index, 'red_avg_words_2guess'] = red_stats['average_expected_guesses']
            df.at[index, 'blue_avg_words_2guess'] = blue_stats['average_expected_guesses']
            df.at[index, 'red_avg_words_guessed'] = red_stats['average_correct_guesses']
            df.at[index, 'blue_avg_words_guessed'] = blue_stats['average_correct_guesses']
            df.at[index, 'reason'] = result[1]
            df.at[index, 'red_turns'] = red_stats['total_hints']
            df.at[index, 'blue_turns'] = blue_stats['total_hints']

        except Exception as e:
            print(f"skipped game between {red_team} and {blue_team}: {e}")

    # Write the updated DataFrame back to the same Excel file
    df.to_excel(file_path, index=False)
    if not already_played:
        print(f"Executed {red_team} vs {blue_team}")


  0%|          | 0/58 [00:00<?, ?it/s]

Executed gpt-4o vs gpt-4o-mini
Executed gpt-4o vs gpt-3.5-turbo
Executed gpt-4o vs grok-beta
Executed gpt-4o vs claude-3-5-sonnet-latest
Executed gpt-4o vs claude-3-5-haiku-latest
Playing gpt-4o vs llama3.2-11b-vision...


 10%|█         | 6/58 [01:49<15:44, 18.17s/it]

Executed gpt-4o vs llama3.2-11b-vision
Playing gpt-4o vs llama3.2-90b-vision...


 12%|█▏        | 7/58 [07:20<1:06:34, 78.33s/it]

...

Executed gpt-3.5-turbo vs gpt-4o
Playing gpt-3.5-turbo vs gpt-4o-mini...


 28%|██▊       | 16/58 [56:31<3:43:21, 319.08s/it]

Executed gpt-3.5-turbo vs gpt-4o-mini
Playing gpt-3.5-turbo vs grok-beta...


 28%|██▊       | 16/58 [59:40<2:36:38, 223.77s/it]


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}